In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [3]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [4]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S

In [5]:
def clean_data(df):
    df['Sex'] = df['Sex'].replace({"male": 0, "female": 1})
    df = pd.get_dummies(df, columns=['Embarked'])
    return df

In [6]:
train_data = clean_data(train_data)
test_data = clean_data(test_data)

In [7]:
X_train = train_data.drop('Survived', axis = 1)
y_train = train_data['Survived']
X_test = test_data

In [8]:
train_data.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    0  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1      0   
2                             Heikkinen, Miss. Laina    1  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1      0   
4                           Allen, Mr. William Henry    0  35.0      0      0   

             Ticket     Fare Cabin  Embarked_C  Embarked_Q  Embarked_S  
0         A/5 21171   7.2500   NaN           0           0           1  
1          PC 17599  71.2833   C85           1           0           0  
2  STON/O2. 3101282   7.9250   NaN           0           0           1  
3            113803  53.1000  C123           0           0           1  
4            373450   8.0500   NaN           0           0           1

In [9]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(891, 13)
(891,)
(418, 13)


In [10]:
X_train[X_train['Age'].isnull()]

PassengerId  Pclass                                      Name  Sex  Age  \
5              6       3                          Moran, Mr. James    0  NaN   
17            18       2              Williams, Mr. Charles Eugene    0  NaN   
19            20       3                   Masselmani, Mrs. Fatima    1  NaN   
26            27       3                   Emir, Mr. Farred Chehab    0  NaN   
28            29       3             O'Dwyer, Miss. Ellen "Nellie"    1  NaN   
..           ...     ...                                       ...  ...  ...   
859          860       3                          Razi, Mr. Raihed    0  NaN   
863          864       3         Sage, Miss. Dorothy Edith "Dolly"    1  NaN   
868          869       3               van Melkebeke, Mr. Philemon    0  NaN   
878          879       3                        Laleff, Mr. Kristo    0  NaN   
888          889       3  Johnston, Miss. Catherine Helen "Carrie"    1  NaN   

     SibSp  Parch      Ticket     Fare Cabin  Embarked_C  Embarked_Q  \
5        0      0      330877   8.4583   NaN           0           1   
17       0      0      244373  13.0000   NaN           0           0   
19       0      0        2649   7.2250   NaN           1           0   
26       0      0        2631   7.2250   NaN           1           0   
28       0      0      330959   7.8792   NaN           0           1   
..     ...    ...         ...      ...   ...         ...         ...   
859      0      0        2629   7.2292   NaN           1           0   
863      8      2    CA. 2343  69.5500   NaN           0           0   
868      0      0      345777   9.5000   NaN           0           0   
878      0      0      349217   7.8958   NaN           0           0   
888      1      2  W./C. 6607  23.4500   NaN           0           0   

     Embarked_S  
5             0  
17            1  
19            0  
26            0  
28            0  
..          ...  
859           0  
863           1  
868           1  
878           1  
888           1  

[177 rows x 13 columns]

In [11]:
X_train['Age'] = X_train['Age'].fillna(X_train['Age'].median())

In [12]:
X_train[X_train['Age'].isnull()]

Empty DataFrame
Columns: [PassengerId, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked_C, Embarked_Q, Embarked_S]
Index: []

In [13]:
X_train = X_train.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis = 1).to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis = 1).to_numpy()

In [14]:
print(X_train.shape)
print(X_test.shape)

(891, 9)
(418, 9)


In [15]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [16]:
def compute_cost_logistic(X, y, w, b, lambda_):
    m, n = X.shape
    cost = 0
    
    for i in range(m):
        z_i = np.dot(X[i], w) + b
        f_wb_i = sigmoid(z_i)
        cost += -y[i] * np.log(f_wb_i) - (1 - y[i]) * np.log(1 - f_wb_i)

    cost /=  m
    
    reg_term = 0
    for j in range(n):
        reg_term += (w[j] ** 2)
    reg_term = reg_term * (lambda_ / (2 * m))
    
    total_cost = cost + reg_term
    
    return total_cost

In [17]:
def compute_gradient_logistic(X, y, w, b, lambda_):
    m,n = X.shape
    
    dj_dw = np.zeros(n,)
    dj_db = 0
    
    for i in range(m):
        z_i = np.dot(X[i], w) + b
        f_wb_i = sigmoid(z_i)
        err_i = f_wb_i - y[i]
        for j in range(n):
            dj_dw[j] += err_i * X[i][j]
        dj_db += err_i
    dj_dw /= m
    dj_db /= m
    
    for j in range(n):
        dj_dw[j] += w[j] * (lambda_ / m)
    
    return dj_dw, dj_db

In [18]:
def gradient_descent(X, y, w_in, b_in, alpha, num_iters, lambda_):
    m = len(X)
    
    J_history = []
    w_history = []
    
    for i in range(num_iters):
        dj_dw, dj_db = compute_gradient_logistic(X, y, w_in, b_in, lambda_)
        
        w_in -= alpha * dj_dw
        b_in -= alpha * dj_db
        
        if i < 100000:
            cost = compute_cost_logistic(X, y, w_in, b_in, lambda_)
            J_history.append(cost)
            
        if i % math.ceil(num_iters / 10) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
            
    return w_in, b_in, J_history, w_history         

In [19]:
def predict(X, w, b):
    m, n = X.shape
    p = np.zeros(m)
    
    for i in range(m):
        z_i = np.dot(X[i], w) + b
        f_wb = sigmoid(z_i)
        
        p[i] = f_wb >= .5
        
    return p

In [20]:
np.random.seed(42)
initial_w = 0.01 * (np.random.rand(9) - 0.5)
initial_b = 0

iterations = 1000
alpha = 0.1e-3

w,b, J_history,_ = gradient_descent(X_train ,y_train, initial_w, initial_b, alpha, iterations, 0)

Iteration    0: Cost     0.71   
Iteration  100: Cost     0.63   
Iteration  200: Cost     0.63   
Iteration  300: Cost     0.62   
Iteration  400: Cost     0.62   
Iteration  500: Cost     0.62   
Iteration  600: Cost     0.62   
Iteration  700: Cost     0.62   
Iteration  800: Cost     0.62   
Iteration  900: Cost     0.62   
Iteration  999: Cost     0.62   


In [21]:
fig, ax = plt.subplots(1,1, figsize = (5,4))

ax.plot(J_history)
ax.set_ylabel("train loss")
ax.set_xlabel("iterations")
plt.show()

In [22]:
p = predict(X_train, w, b)
print('Train Accuracy: %f'%(np.mean(p == y_train) * 100))

Train Accuracy: 66.891134


In [24]:
p = predict(X_test, w, b)

In [25]:
p.shape

(418,)

In [26]:
output = np.c_[test_data["PassengerId"].to_numpy(), p].astype(int)

In [27]:
output = pd.DataFrame(output, columns = ['PassengerId', 'Survived'])

In [28]:
output.to_csv("logistic_submission.csv", index = False)